In [1]:
import urllib2
import urlparse
import re

def crawlWebpage(url):
    connection = urllib2.urlopen(url)
    page = connection.read()
    connection.close()
    return page

def formatText(text):
    trimmed = text.replace('\n', ' ')
    trimmed = trimmed.replace('\t', ' ')
    trimmed = trimmed.replace('_', ' ')
    trimmed = re.sub(r'[^\w]', ' ', trimmed)
    trimmed = " ".join(trimmed.split())
    return trimmed
    

def isValidText(text):
    trimmed = text.replace('\n', ' ')
    trimmed.replace('\t', ' ')
    trimmed = " ".join(trimmed.split())
    if len(text) > 1:
        return True
    for i in '. /!@#$%^&*()<>,.`~·–':
        if text == i:
            return False
    return text != ''

def getXPath(text, tree):
    real_text = tree.getpath(text.getparent())
    if 'div' not in real_text:
        return real_text
    tags = real_text.split('/')
    num_tags = len(tags)
    currentElement = text.getparent()
    ident = currentElement.get('id')
    count = 0
    while ident == None and count < num_tags:
        currentElement = currentElement.getparent()
        if currentElement == None:
            break
        ident = currentElement.get('id')
        count += 1
    if ident == None:
        return real_text
    else:
        real_text = '//*[@id="%s"]' %ident
        for i in xrange(num_tags-count, num_tags):
            tag = tags[i]
            if 'tr' in tag and tag != 'strong':
                if i > 0:
                    if 'tbody' not in tags[i-1]:
                        tag = 'tbody/' + tag
                else:
                    tag = 'tbody/' + tag
            real_text += '/' + tag
    return real_text
    

def parse_page_for_xpaths(page):
    from cStringIO import StringIO
    from lxml import etree
    
    #This is a touple array of (text, xpath)
    xpaths = []
    
    parser = etree.HTMLParser(remove_blank_text=True)
    tree   = etree.parse(StringIO(page), parser)
    find_text = etree.XPath("//text()")

    for text in find_text(tree):
        formatted_text = formatText(text)
        if isValidText(formatted_text):
            xpath = getXPath(text, tree)
            if 'script' not in xpath and 'body' in xpath:
                xpaths.append((formatted_text, xpath))
    return xpaths

url = 'http://stackoverflow.com/questions/39420152/scraping-font-size-from-html-and-css'
page = crawlWebpage(url)

#This is an array of touple(text, xpath)
xpaths = parse_page_for_xpaths(page)
for i in xrange(0, 50):
    text, xpath = xpaths[i]
    print xpath + " ~> " + text

/html/body/div[3]/div/div[1]/div/div[1]/h3/a ~> current community
/html/body/div[3]/div/div[1]/div/div[2]/ul/li[1]/div/a ~> chat
/html/body/div[3]/div/div[1]/div/div[2]/ul/li[1]/a/div ~> Stack Overflow
/html/body/div[3]/div/div[1]/div/div[2]/ul/li[2]/a/div ~> Meta Stack Overflow
/html/body/div[3]/div/div[1]/div/div[5]/h3/a ~> more stack exchange communities
/html/body/div[3]/div/div[1]/div/div[5]/a ~> company blog
/html/body/div[3]/div/div[2]/a[1]/span ~> Stack Exchange
/html/body/div[3]/div/div[2]/a[2]/span[1] ~> Inbox
/html/body/div[3]/div/div[2]/a[3]/span[1] ~> Reputation and Badges
/html/body/div[3]/div/div[3]/div[1]/span/a[1] ~> sign up
/html/body/div[3]/div/div[3]/div[1]/span/a[2] ~> log in
/html/body/div[3]/div/div[3]/div[1]/span/a[3] ~> tour
/html/body/div[3]/div/div[3]/div[1]/span/a[4] ~> help
/html/body/div[3]/div/div[3]/div[1]/span/div/div/ul/li[1]/a ~> Tour
/html/body/div[3]/div/div[3]/div[1]/span/div/div/ul/li[1]/a/span ~> Start here for a quick overview of the site
/html/

In [2]:
import sys
import time

def get_xpath_fontSize(xpath):
    if '/h6/' in xpath:
        return .83 * 16
    if '/h5/' in xpath:
        return .75 * 16
    if '/h3/' in xpath:
        return 1.17 * 16
    if '/h2/' in xpath:
        return 1.5 * 16
    if '/h1/' in xpath:
        return 2 * 16
    return 16

def get_xpath_data(xpaths):
    xpath_data = []
    for i in xpaths:
        text, xpath = i
        xpath_data.append((text, get_xpath_fontSize(xpath)))
    return xpath_data


#url = 'http://stackoverflow.com/questions/39420152/scraping-font-size-from-html-and-css'
#page = crawlWebpage(url)

#This is an array of touple(text, xpath)
#xpaths = parse_page_for_xpaths(page)

#this is an array of touple(text, fontSize)
xpath_data = get_xpath_data(xpaths)
for i in xrange(0, min(len(xpath_data), 50)):
    text, fontSize = xpath_data[i]
    print fontSize, " ~> " + text
#//*[@id="sidebar"]/div[6]/div/div[1]/a[2]

18.72  ~> current community
16  ~> chat
16  ~> Stack Overflow
16  ~> Meta Stack Overflow
18.72  ~> more stack exchange communities
16  ~> company blog
16  ~> Stack Exchange
16  ~> Inbox
16  ~> Reputation and Badges
16  ~> sign up
16  ~> log in
16  ~> tour
16  ~> help
16  ~> Tour
16  ~> Start here for a quick overview of the site
16  ~> Help Center
16  ~> Detailed answers to any questions you might have
16  ~> Meta
16  ~> Discuss the workings and policies of this site
16  ~> About Us
16  ~> Learn more about Stack Overflow the company
16  ~> Business
16  ~> Learn more about hiring developers or posting ads with us
16  ~> up vote
16  ~> 0
16  ~> down vote
16  ~> favorite
16  ~> I am trying to scrape the font size of each section of text in an HTML page I have spent the past few days trying to do it but I feel like I am trying to re invent the wheel I have looked at python libraries like cssutils beautiful soup but haven t had much luck sadly I have made my own html parser that finds the f

In [8]:
def normalize_xpath_data(xpath_data, print_data = False):
    maxFontSize = 0
    minFontSize = 10000000#just a really high number...fix later
    for i in xpath_data:
        text, fontSize = i
        fontSize = float(fontSize)
        if maxFontSize < fontSize:
            maxFontSize = fontSize
        if minFontSize > fontSize:
            minFontSize = fontSize
    normalized_data = []
    for i in xpath_data:
        text, fontSize = i#fontSize is actually text. Convert it to a float
        trueFontSize = float(fontSize)
        div = float(maxFontSize - minFontSize)
        if div == 0:
            div = 1
        normalizedFontSize = int(round(float(trueFontSize - minFontSize)/div * 6))
        if print_data:
            print fontSize, ' ~> %i'%normalizedFontSize
        normalized_data.append((text, normalizedFontSize))
    return normalized_data
    
#url = 'http://stackoverflow.com/questions/39420152/scraping-font-size-from-html-and-css'
#page = crawlWebpage(url)

#This is an array of touple(text, xpath)
#xpaths = parse_page_for_xpaths(page)

#this is an array of touple(text, fontSize)
#xpath_data = retrieve_xpath_data(url, xpaths)

normalized_fSize = normalize_xpath_data(xpath_data, print_data = True)

18.72  ~> 6
16  ~> 0
16  ~> 0
16  ~> 0
18.72  ~> 6
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  ~> 0
16  

In [4]:
import time
def isCapital(word):
    return not word.islower()

def breakTextIntoWordBundles(xpath_data, print_data = False):
    wordCount = 0
    words = []
    longestWordLen = 0
    longestWord = ''
    maxWordCount = 2**12-1
    for i in xpath_data:
        text, fontSize = i
        textWords = text.split(' ')
        for word in textWords:
            capital = isCapital(word)
            word = word.lower()
            words.append((word, capital, fontSize, min(wordCount, maxWordCount)))
            wordCount += 1
            if len(word) > longestWordLen:
                longestWordLen = len(word)
                longestWord = word
    if print_data:
        start = time.clock()
        print start
        print 'Word Count:       %i' %wordCount
        print 'Longest Word: %s(%i)' %(longestWord, longestWordLen)
        print '========================================='
        for wordBundle in words:
            word, capital, fSize, position = wordBundle
            tempWord = word
            spacesNeeded = longestWordLen - len(word)
            for i in xrange(0, spacesNeeded):
                tempWord += ' '
            print tempWord + '\tcap: %r\tfSize: %i\tpos: %i'%(capital, fSize, position) 
        stop = time.clock()
        print stop
        print stop-start, "seconds"
    return words

#url = 'http://stackoverflow.com/questions/39420152/scraping-font-size-from-html-and-css'
#page = crawlWebpage(url)

#This is an array of touple(text, xpath)
#xpaths = parse_page_for_xpaths(page)

#this is an array of touple(text, fontSize)
#xpath_data = retrieve_xpath_data(url, xpaths)

#Font sizes are now normalized with the rest of the document
#normalized_fSize = normalize_xpath_data(xpath_data)

#Array of touple(lowercase word, capital, normalizedFontSize, position)
wordBundles = breakTextIntoWordBundles(normalized_fSize, print_data = True)

0.821225
Word Count:       408
Longest Word: expressionengine(16)
current         	cap: False	fSize: 6	pos: 0
community       	cap: False	fSize: 6	pos: 1
chat            	cap: False	fSize: 0	pos: 2
stack           	cap: True	fSize: 0	pos: 3
overflow        	cap: True	fSize: 0	pos: 4
meta            	cap: True	fSize: 0	pos: 5
stack           	cap: True	fSize: 0	pos: 6
overflow        	cap: True	fSize: 0	pos: 7
more            	cap: False	fSize: 6	pos: 8
stack           	cap: False	fSize: 6	pos: 9
exchange        	cap: False	fSize: 6	pos: 10
communities     	cap: False	fSize: 6	pos: 11
company         	cap: False	fSize: 0	pos: 12
blog            	cap: False	fSize: 0	pos: 13
stack           	cap: True	fSize: 0	pos: 14
exchange        	cap: True	fSize: 0	pos: 15
inbox           	cap: True	fSize: 0	pos: 16
reputation      	cap: True	fSize: 0	pos: 17
and             	cap: False	fSize: 0	pos: 18
badges          	cap: True	fSize: 0	pos: 19
sign            	cap: False	fSize: 0	pos: 20
up       

In [9]:
#put it all together!
def parsePlainHits(url, print_data = False):
    page = crawlWebpage(url)

    #This is an array of touple(text, xpath)
    xpaths = parse_page_for_xpaths(page)

    #this is an array of touple(text, fontSize)
    xpath_data = get_xpath_data(xpaths)

    #Font sizes are now normalized with the rest of the document
    normalized_fSize = normalize_xpath_data(xpath_data)

    #Array of touple(lowercase word, capital, normalizedFontSize, position)
    wordBundles = breakTextIntoWordBundles(normalized_fSize, print_data)
    return wordBundles
parsedPlainHits = parsePlainHits('https://www.crummy.com/software/BeautifulSoup/bs4/doc/', print_data = True)

1.592934
Word Count:       397
Longest Word: navigablestring(15)
navigation     	cap: True	fSize: 0	pos: 0
index          	cap: False	fSize: 0	pos: 1
beautiful      	cap: True	fSize: 0	pos: 2
soup           	cap: True	fSize: 0	pos: 3
4              	cap: True	fSize: 0	pos: 4
4              	cap: True	fSize: 0	pos: 5
0              	cap: True	fSize: 0	pos: 6
documentation  	cap: False	fSize: 0	pos: 7
parser         	cap: True	fSize: 0	pos: 8
typical        	cap: True	fSize: 0	pos: 9
usage          	cap: False	fSize: 0	pos: 10
advantages     	cap: True	fSize: 0	pos: 11
disadvantages  	cap: True	fSize: 0	pos: 12
python         	cap: True	fSize: 0	pos: 13
s              	cap: False	fSize: 0	pos: 14
html           	cap: False	fSize: 0	pos: 15
parser         	cap: False	fSize: 0	pos: 16
beautifulsoup  	cap: True	fSize: 0	pos: 17
markup         	cap: False	fSize: 0	pos: 18
html           	cap: False	fSize: 0	pos: 19
parser         	cap: False	fSize: 0	pos: 20
batteries      	cap: True	fSize: 

/Users/noahziems/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:27: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [10]:
#Now we multi thread it!
from multiprocessing.dummy import Pool as ThreadPool 

urls = [
  'http://www.python.org', 
  'http://www.python.org/about/',
  'http://www.onlamp.com/pub/a/python/2003/04/17/metaclasses.html',
  'http://www.python.org/doc/',
  'http://www.python.org/download/',
  'http://www.python.org/getit/',
  'http://www.python.org/community/',
  'https://wiki.python.org/moin/',
  ]
pool = ThreadPool(8)

start = time.clock()
results = pool.map(parsePlainHits, urls)

#close the pool and wait for the work to finish 
pool.close() 
pool.join() 
stop = time.clock()
print stop-start, 'seconds'

0.286923 seconds


In [11]:
print results

[[('industrial', True, 0, 0), ('light', True, 0, 1), ('magic', True, 0, 2), ('runs', True, 0, 3), ('on', False, 0, 4), ('python', True, 0, 5), ('by', False, 0, 6), ('tim', True, 0, 7), ('fortenberry', True, 0, 8)], [], [('content', True, 0, 0), ('all', True, 0, 1), ('articles', True, 0, 2), ('python', True, 0, 3), ('news', True, 0, 4), ('numerically', True, 0, 5), ('python', True, 0, 6), ('python', True, 0, 7), ('xml', True, 0, 8), ('python', True, 0, 9), ('topics', True, 0, 10), ('community', True, 0, 11), ('database', True, 0, 12), ('distributed', True, 0, 13), ('education', True, 0, 14), ('getting', True, 0, 15), ('started', True, 0, 16), ('graphics', True, 0, 17), ('internet', True, 0, 18), ('os', True, 0, 19), ('programming', True, 0, 20), ('scientific', True, 0, 21), ('tools', True, 0, 22), ('tutorials', True, 0, 23), ('user', True, 0, 24), ('interfaces', True, 0, 25), ('onlamp', True, 0, 26), ('subjects', True, 0, 27), ('linux', True, 0, 28), ('apache', True, 0, 29), ('mysql', T

In [ ]:
import os
import sys
sys.path.append(os.path.abspath('/Users/noahziems/PycharmProjects/BackRub/'))
import Lexicon as lex
import struct

def encodeHit(cap, fSize, pos):
    result = 0
    result += int(cap) << 15
    result += ((fSize << 12) & 0b0111000000000000)
    if pos > 0b111111111111:
        pos = 0b111111111111
    result += (pos & 0b0000111111111111)
    return struct.pack('H', result)

def encodePlainHits(parsedPlainHits, print_data = False):
    hitLists = {}
    words = {}
    for i in parsedPlainHits:
        word, cap, fSize, pos = i
        wordID = lex.getID(word)
        words[wordID] = word
        if wordID in hitLists:
            hitLists[wordID] = hitLists[wordID] + encodeHit(cap, fSize, pos)
        else:
            hitLists[wordID] = encodeHit(cap, fSize, pos)
    if print_data:
        for hitListKey in hitLists.keys():
            word = words[hitListKey]
            length = len(hitLists[hitListKey])
            print word + '[%i]\t[%i hits]\t[%i bytes]:' %(hitListKey,  length/2, length)
            print hitLists[hitListKey] + "\n"
    return hitLists

lex.load()
encodedPlainHits = encodePlainHits(parsedPlainHits, print_data = True)
sys.getsizeof(encodedPlainHits)
lex.save()